In [2]:
# !python -m spacy download 'en_core_web_lg'

In [3]:
# !wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip

In [4]:
# !unzip /content/dataset52a7b21.zip

In [5]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [6]:
import pandas as pd
import csv

In [7]:
# file_path = './dataset/small_test.csv'
file_path = '/drive/My Drive/colab_data/train_100k.csv'
# file_path = '/content/dataset/train.csv'
# escape_char_defined = "\\"
train_data = pd.read_csv(file_path, index_col=0)
# train_data = pd.read_csv(file_path, error_bad_lines=False, quoting=csv.QUOTE_NONE, escapechar=escape_char_defined)
train_data.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
1320140,75 Pcs Rainbow Sia Seeds Flower Indoor Flower ...,High Quality Seeds with Germination rate of ab...,[High Quality Seeds with Germination rate of a...,Generic,1988
2271489,Ozar Adjustable Hand Reamers (Multicolor),NaN,"[Brand: ozar,Hand reamers,3/a 13/32 inch, 7/16...",Ozar,740325
1199120,Bastar Art Brass 3 Rider Figurine (25 cm x 8 c...,"Dhokra Art, A Genuine Indian Work Of Art Produ...",[Material: Brass; A Genuine Indian Work Of Art...,Bastar Art,251
2299664,UCB United Colors of Benetton Men's Dreams Go ...,United dreams goes a step further by creating ...,[The new men's fragrance from the united dream...,United Colors of Benetton,76
1942941,"Yihome Womens Coats, Pure Color Faux Fur Pocke...",<br><b>Product information:</b><br>Season:All ...,"[√ It is made of high quality materials, durab...",Yihome,9975


In [8]:
train_data = train_data.dropna(axis=0)

In [9]:
import numpy as np
import spacy

# Need to load the large model to get the vectors
nlp = spacy.load('en_core_web_lg')

In [10]:
def get_vec(col, df):
  lst=[]
  with nlp.disable_pipes():
    for idx, data in df.iterrows():
      filtered_text = ""
      doc = nlp(data[col])
      for token in doc:
          if(token.is_stop==False):
              filtered_text = filtered_text+" "+str(token)
      lst.append(nlp(filtered_text).vector)
    return np.array(lst)

In [11]:
# desc_vectors = get_vec("DESCRIPTION", train_data)
# desc_vectors.shape

In [12]:
# title_vectors = get_vec("TITLE", train_data)
# title_vectors.shape

In [13]:
# brand_vectors = get_vec("BRAND", train_data)
# brand_vectors.shape

In [14]:
bp_vectors = get_vec("BULLET_POINTS", train_data)
bp_vectors.shape

(72695, 300)

In [15]:
# A = (title_vectors, desc_vectors, brand_vectors, bp_vectors)
# X = np.hstack(A)
# X.shape

In [16]:
from sklearn.svm import LinearSVC
# Create the LinearSVC model
model = LinearSVC(random_state=1, dual=False)
# Fit the model
model.fit(bp_vectors, train_data["BROWSE_NODE_ID"])

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=1, tol=0.0001,
          verbose=0)

In [17]:
# file_path = './dataset/small_test.csv'
file_path = '/content/dataset/test.csv'
escape_char_defined = "\\"
# data = pd.read_csv(file_path, error_bad_lines=False)
test_data = pd.read_csv(file_path, error_bad_lines=False, quoting=csv.QUOTE_NONE, escapechar=escape_char_defined)
test_data.head()

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero


In [18]:
test_data = test_data.replace(np.nan, '', regex=True)

In [19]:
# test_data = test_data.head()

In [20]:
# desc_vectors = get_vec("DESCRIPTION", test_data)
# desc_vectors.shape

In [21]:
# title_vectors = get_vec("TITLE", test_data)
# title_vectors.shape

In [22]:
# brand_vectors = get_vec("BRAND", test_data)
# brand_vectors.shape

In [23]:
bp_vectors = get_vec("BULLET_POINTS", test_data)
bp_vectors.shape

(110775, 300)

In [24]:
# A = (title_vectors, desc_vectors, brand_vectors, bp_vectors)
# X = np.hstack(A)
# X.shape

In [25]:
preds = model.predict(bp_vectors)
preds

array([1140,   69,  593, ...,  800,  800,  800])

In [ ]:
df = pd.DataFrame()
df["PRODUCT_ID"] = test_data["PRODUCT_ID"]
df["BROWSE_NODE_ID"] = preds
df

In [ ]:
df.to_csv('/drive/My Drive/colab_data/submit_100k.csv',index=False)
